In [2]:
!pip install esgf-pyclient

!pip install pytest
 
!pip install flake8
 
!pip install sphinx
 
!pip install nbsphinx
 
!pip install pandoc
 
!pip install ipykernel
 
!pip install nbconvert
 
!pip install defusedxml
 
!pip install jinja2
 
!pip install requests
 
!pip install webob
 
!pip install myproxyclient
 
!pip install nbval
!pip install requests_cache==0.5
!pip install xarray

!pip install pydap
!pip install netcdf4
!pip install shapely
!pip install pydap

!pip install spicy

In [133]:
from pyesgf.search import SearchConnection
conn = SearchConnection('https://esgf-data.dkrz.de/esg-search', distrib=True)

In [134]:
ctx = conn.new_context(
    latest=True,
    project='CMIP5',
    model='MRI-CGCM3',
    experiment='rcp45',
    variable=['pr','tasmax','tasmin'],
    time_frequency="day",
    realm='atmos',
    ensemble='r1i1p1',
    data_node='esgf-data1.ceda.ac.uk')
ctx.hit_count

1

In [135]:
ctx.facet_counts

{'Conventions': {},
 'access': {'OPENDAP': 1, 'HTTPServer': 1, 'GridFTP': 1, 'Globus': 1},
 'activity': {},
 'activity_drs': {},
 'activity_id': {},
 'amodell': {},
 'branch_method': {},
 'cera_acronym': {},
 'cf_standard_name': {'wind_speed': 1,
  'toa_outgoing_longwave_flux': 1,
  'surface_upwelling_shortwave_flux_in_air': 1,
  'surface_upwelling_longwave_flux_in_air': 1,
  'surface_upward_sensible_heat_flux': 1,
  'surface_upward_latent_heat_flux': 1,
  'surface_downwelling_shortwave_flux_in_air': 1,
  'surface_downwelling_longwave_flux_in_air': 1,
  'specific_humidity': 1,
  'snowfall_flux': 1,
  'relative_humidity': 1,
  'precipitation_flux': 1,
  'northward_wind': 1,
  'lagrangian_tendency_of_air_pressure': 1,
  'geopotential_height': 1,
  'eastward_wind': 1,
  'convective_precipitation_flux': 1,
  'cloud_area_fraction': 1,
  'air_temperature': 1,
  'air_pressure_at_sea_level': 1},
 'cmor_table': {'day': 1},
 'contact': {},
 'data_node': {'esgf-data1.ceda.ac.uk': 1},
 'data_specs

In [136]:
result = ctx.search()[0]
result.dataset_id

'cmip5.output1.MRI.MRI-CGCM3.rcp45.day.atmos.day.r1i1p1.v20120701|esgf-data1.ceda.ac.uk'

In [137]:
urls = []
files = result.file_context().search()
for file in files:
  for variable in ['/tasmin/', '/tasmax/', '/pr/']:
    if variable in file.opendap_url:
          tasmax_url = file.opendap_url
          urls.append(tasmax_url)

In [138]:
len(urls)
urls

['http://esgf-data1.ceda.ac.uk/thredds/dodsC/esg_dataroot/cmip5/output1/MRI/MRI-CGCM3/rcp45/day/atmos/day/r1i1p1/v20120701/pr/pr_day_MRI-CGCM3_rcp45_r1i1p1_20060101-20151231.nc',
 'http://esgf-data1.ceda.ac.uk/thredds/dodsC/esg_dataroot/cmip5/output1/MRI/MRI-CGCM3/rcp45/day/atmos/day/r1i1p1/v20120701/pr/pr_day_MRI-CGCM3_rcp45_r1i1p1_20160101-20251231.nc',
 'http://esgf-data1.ceda.ac.uk/thredds/dodsC/esg_dataroot/cmip5/output1/MRI/MRI-CGCM3/rcp45/day/atmos/day/r1i1p1/v20120701/pr/pr_day_MRI-CGCM3_rcp45_r1i1p1_20260101-20351231.nc',
 'http://esgf-data1.ceda.ac.uk/thredds/dodsC/esg_dataroot/cmip5/output1/MRI/MRI-CGCM3/rcp45/day/atmos/day/r1i1p1/v20120701/pr/pr_day_MRI-CGCM3_rcp45_r1i1p1_20360101-20451231.nc',
 'http://esgf-data1.ceda.ac.uk/thredds/dodsC/esg_dataroot/cmip5/output1/MRI/MRI-CGCM3/rcp45/day/atmos/day/r1i1p1/v20120701/pr/pr_day_MRI-CGCM3_rcp45_r1i1p1_20460101-20551231.nc',
 'http://esgf-data1.ceda.ac.uk/thredds/dodsC/esg_dataroot/cmip5/output1/MRI/MRI-CGCM3/rcp45/day/atmos/day

In [154]:
import xarray as xr
ds = xr.open_dataset(urls[29])
print(ds)

<xarray.Dataset>
Dimensions:    (time: 1826, bnds: 2, lat: 160, lon: 320)
Coordinates:
  * time       (time) datetime64[ns] 2096-01-01T12:00:00 ... 2100-12-31T12:00:00
  * lat        (lat) float64 -89.14 -88.03 -86.91 -85.79 ... 86.91 88.03 89.14
  * lon        (lon) float64 0.0 1.125 2.25 3.375 ... 355.5 356.6 357.8 358.9
    height     float64 ...
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) datetime64[ns] ...
    lat_bnds   (lat, bnds) float64 ...
    lon_bnds   (lon, bnds) float64 ...
    tasmin     (time, lat, lon) float32 ...
Attributes: (12/28)
    institution:                     MRI (Meteorological Research Institute, ...
    institute_id:                    MRI
    experiment_id:                   rcp45
    source:                          MRI-CGCM3 2011 atmosphere: GSMUV (gsmuv-...
    model_id:                        MRI-CGCM3
    forcing:                         GHG, SA, Oz, LU, Sl, Vl, BC, OC (GHG inc...
    ...                         

In [155]:
import pandas as pd
import netCDF4 as nc
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

In [156]:
bangui = pd.read_excel('Bangui_pts.xlsx', index_col=0, dtype={'id': int, 'lat': float, 'long': float})
banguiValues = []
for row in bangui.iterrows():
  banguiValues.append([row[1].long, row[1].lat])
  
polygonBangui = Polygon(banguiValues)

In [157]:
long_df = ds.lon.to_dataframe()

In [158]:
lat_df = ds.lat.to_dataframe()

In [159]:
lat_df

,height,lat
lat,,
-89.14152,2.0,-89.14152
-88.02943,2.0,-88.02943
-86.91077,2.0,-86.91077
-85.79063,2.0,-85.79063
-84.66992,2.0,-84.66992
...,...,...
84.66992,2.0,84.66992
85.79063,2.0,85.79063
86.91077,2.0,86.91077


In [160]:
long_df

,height,lon
lon,,
0.000,2.0,0.000
1.125,2.0,1.125
2.250,2.0,2.250
3.375,2.0,3.375
4.500,2.0,4.500
...,...,...
354.375,2.0,354.375
355.500,2.0,355.500
356.625,2.0,356.625


In [169]:
long_lat_df = []
long_bangui = []
lat_bangui = []
for i, long  in  long_df.iterrows():
    for j, lati  in  lat_df.iterrows():
        if polygonBangui.contains(Point(long.lon, lati.lat)):
            long_bangui.append(long.lon)
            lat_bangui.append(lati.lat)
            long_lat_df.append({'lon': long.lon, 'lat': lati.lat})
        
long_lat_df
pd.DataFrame(long_lat_df)

,lon,lat
0,18.000,6.16819
1,19.125,5.04670
2,20.250,5.04670
3,20.250,7.28968
4,21.375,5.04670
5,22.500,3.92521
6,23.625,3.92521
7,24.750,3.92521
8,25.875,3.92521
9,27.000,2.80372


In [82]:
long_lat_df = []
for i, long  in  long_df.iterrows():
    for j, lati  in  lat_df.iterrows():
        long_lat_df.append({'lon': long.lon, 'lat': lati.lat})
        
long_lat_df
pd.DataFrame(long_lat_df)

,lon,lat
0,0.000,-89.14152
1,0.000,-88.02943
2,0.000,-86.91077
3,0.000,-85.79063
4,0.000,-84.66992
...,...,...
51195,358.875,84.66992
51196,358.875,85.79063
51197,358.875,86.91077
51198,358.875,88.02943


In [83]:
essaka = pd.read_excel('essaka_pts.xlsx', index_col=0, dtype={'id': int, 'lat': float, 'long': float})
essakaValues = []
for row in essaka.iterrows():
  essakaValues.append([row[1].long, row[1].lat])
  
polygonessaka = Polygon(essakaValues)

In [96]:
long_lat_array= []
for i, long  in  long_df.iterrows():
    for j, lati  in  lat_df.iterrows():
        if polygonessaka.contains(Point(long.lon, lati.lat)):
            long_lat_array.append({'lon': long.lon, 'lat': lati.lat})
        
long_lat_df = pd.DataFrame(long_lat_array)

In [97]:
long_lat_df

,lon,lat
0,19.125,-11.77564
1,19.125,-3.92521
2,20.250,-10.65415
3,20.250,-9.53266
4,20.250,-8.41117
5,20.250,-7.28968
6,20.250,-6.16819
7,20.250,-5.04670
8,21.375,-10.65415
9,21.375,-9.53266


In [162]:
ds

<xarray.Dataset>
Dimensions:    (time: 1826, bnds: 2, lat: 160, lon: 320)
Coordinates:
  * time       (time) datetime64[ns] 2096-01-01T12:00:00 ... 2100-12-31T12:00:00
  * lat        (lat) float64 -89.14 -88.03 -86.91 -85.79 ... 86.91 88.03 89.14
  * lon        (lon) float64 0.0 1.125 2.25 3.375 ... 355.5 356.6 357.8 358.9
    height     float64 ...
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) datetime64[ns] ...
    lat_bnds   (lat, bnds) float64 ...
    lon_bnds   (lon, bnds) float64 ...
    tasmin     (time, lat, lon) float32 ...
Attributes: (12/28)
    institution:                     MRI (Meteorological Research Institute, ...
    institute_id:                    MRI
    experiment_id:                   rcp45
    source:                          MRI-CGCM3 2011 atmosphere: GSMUV (gsmuv-...
    model_id:                        MRI-CGCM3
    forcing:                         GHG, SA, Oz, LU, Sl, Vl, BC, OC (GHG inc...
    ...                              ...
    title:                           MRI-CGCM3 model output prepared for CMIP...
    parent_experiment:               historical
    modeling_realm:                  atmos
    realization:                     1
    cmor_version:                    2.6.0
    DODS_EXTRA.Unlimited_Dimension:  time

In [184]:
i = ds['tasmin'].sel(lon=long_bangui, lat=lat_bangui).to_dataframe()
i['tasmin'] = i['tasmin'] - 273.15

In [186]:
i.to_csv('h.csv', header=True)

In [187]:
import pandas as pd
df = pd.read_csv('h.csv')


In [192]:
df[410835:410849]

,time,lat,lon,height,tasmin
410835,2100-12-31 12:00:00,2.80372,18.000,2.0,20.066064
410836,2100-12-31 12:00:00,2.80372,19.125,2.0,20.248560
410837,2100-12-31 12:00:00,2.80372,20.250,2.0,20.027948
410838,2100-12-31 12:00:00,2.80372,20.250,2.0,20.027948
410839,2100-12-31 12:00:00,2.80372,21.375,2.0,19.805109
410840,2100-12-31 12:00:00,2.80372,22.500,2.0,19.095575
410841,2100-12-31 12:00:00,2.80372,23.625,2.0,19.206262
410842,2100-12-31 12:00:00,2.80372,24.750,2.0,19.831293
410843,2100-12-31 12:00:00,2.80372,25.875,2.0,19.060632
410844,2100-12-31 12:00:00,2.80372,27.000,2.0,17.834039


In [ ]:
import requests as r
r.get("http://esg-datanode.jpl.nasa.gov/esg-search/search?facets=model,experiment,project&limit=0")

ConnectionError: ignored